# Crop Yield Analysis: Exploring Trends and Predictive Models

### I. Introduction

##### Objective and Scope

The objective of this project is to analyze the crop yield data of Rural Municipalities (RM) in Saskatchewan, Canada from 1938 to 2021. The dataset contains yield data for various crops, including Winter Wheat, Canola, Spring Wheat, Mustard, Durum, Sunflowers, Oats, Lentils, Peas, Barley, Fall Rye, Canary Seed, Spring Rye, Tame Hay, Flax, and Chickpeas.

The scope of the project is to perform a thorough data analysis on the yield data, including data cleaning, exploration, and visualization. Machine learning techniques will be applied to develop models for predicting crop yields. The analysis will focus on identifying trends and patterns in the data and providing insights into factors that affect crop yields in different regions of Saskatchewan.

The dataset covers crop yield data for 299 Rural Municipalities (RM) in Saskatchewan, Canada. The analysis will cover the entire range of years in the dataset, from 1938 to 2021.

#### Research questions and Hypotheses

Research questions:

* What is the overall trend in crop yields in Saskatchewan from 1938 to 2021?
* Which crops have the highest and lowest yields in Saskatchewan, and how have these yields changed over time?
* Are there any differences in crop yields between different regions of Saskatchewan?

Hypotheses:

* There is an overall increasing or decreasing trend in crop yields in Saskatchewan over time.
* Certain crops may have higher or lower yields than others, and these yields may change at different rates over time.
* There may be significant differences in crop yields between different regions of Saskatchewan.

#### Description of the dataset and its sources

##### *Load Library*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import geopandas as gpd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import skew, kurtosis
from sklearn.cluster import KMeans

from statsmodels.tsa.arima.model import ARIMA

plt.rcParams['figure.figsize'] = (30, 20)

import warnings
warnings.filterwarnings("ignore")

#### *Description*

In [ ]:
# Load spacial dataset by reading 'RuralMunicipality/Rural Municipality.shp'
gdf = gpd.read_file('RuralMunicipality/Rural Municipality.shp')
gdf

In [ ]:
gdata = gdf.copy()
gdata

In [ ]:
gdata.rename(columns={"RMNO": "RM"}, inplace=True)
gdata

In [ ]:
# Load dataset by reading 'rm_crop_yields_1938_2021.csv' 
df = pd.read_csv('rm_crop_yields_1938_2021.csv')
df

In [ ]:
# Make a copy of orginial dataset, we will do all modifications on the copy.
data = df.copy()
data

In [ ]:
# Check the shape of the dataset.
data.shape

In [ ]:
# Take a look at the first 5 rows of the dataset.
data.head()

In [ ]:
# Take a look at the last 5 rows of the dataset.
data.tail()

In [ ]:
# Check the number of unique value for 'Year' and 'RM' column
print('Year:' + str(data['Year'].nunique()))
print('RM:' + str(data['RM'].nunique()))

In [ ]:
# Show information of the dataset.
data.info()


The dataset used for this project is named "rm_crop_yields_1938_2021.csv" and contains yield data for different crops grown in different RMs (Rural Municipalities) in Saskatchewan, Canada from 1938 to 2021. The dataset contains 25017 observations and 18 features. The features include the year and RM number along with the yield data for 16 different crops including Winter Wheat, Canola, Spring Wheat, Mustard, Durum, Sunflowers, Oats, Lentils, Peas, Barley, Fall Rye, Canary Seed, Spring Rye, Tame Hay, Flax, and Chickpeas.

Out of the 16 crops, not all are present in every year for every RM, hence some of the yield values are missing. The data types of the features are either float64 or int64. The numerical features in the dataset represent yield data and are measured in bushels per acre (bu/acre).

The time period covered by the dataset is from 1938 to 2021, giving a total of 84 years of data.

Unfortunately, the source of this dataset is not specified, and we do not have any information regarding its origin or collection process. However, the dataset appears to be reliable and of high quality, and we will conduct a thorough analysis of the data to derive meaningful insights.

It is important to note that this dataset covers crop yield data only for Saskatchewan, Canada and may not be generalizable to other regions or countries.

In [ ]:
gdata.info()

In [ ]:
gdata.isna().sum()

In [ ]:
gdata['RM'] = gdata['RM'].astype(int)
gdata['RMNM'] = gdata['RMNM'].astype('string')
gdata['PPID'] = gdata['PPID'].astype('string')
gdata['EFFDT'] = gdata['EFFDT'].astype('string')

## II. Data Preprocessing

#### *Data Cleaning*

In [ ]:
# To do summary statistics of the dataset. 
data.describe().T

##### Missing values

In [ ]:
#Check missing values

data.isnull().sum()

#### *Imutate the missing values.*
In our case, we have a large number of missing values in some columns, so dropping the missing values may result in significant data loss. Therefore, we will use imputation to fill in the missing values.

There are different strategies for imputation, and we will use a simple strategy of filling in the missing values with the mean value of the corresponding column. We can use the fillna method to fill in the missing values with the mean value of the column.

In [ ]:
data.fillna(data.mean(), inplace=True)

In [ ]:
data.describe().T

##### *Check outliers*

In [ ]:
# Create a figure with 8 rows and 2 columns
fig, axes = plt.subplots(nrows=8, ncols=2)

# Loop through each crop and create a boxplot in the corresponding axis
crops = list(data.columns[2:])
for i, crop in enumerate(crops):
    row = i // 2
    col = i % 2
    sns.boxplot(x=data[crop], ax=axes[row, col], orient='v', color='lightblue')
    axes[row, col].set_title(crop)
    
# Remove the empty subplot(s)
if len(crops) % 2 != 0:
    fig.delaxes(axes[-1, -1])
    
# Adjust the spacing between the subplots
fig.subplots_adjust(hspace=0.5)
    
plt.show()

Those box plots show clearly that most of columns have outliers exist. However, the analysis is focused on identifying trends and patterns in the data, it would be best to keep the outliers in the dataset. 

#### *Identify outliers using z-scores*

In [ ]:
# define a function to identify outliers using z-scores
def detect_outliers_zscore(data, threshold=3):
    z_scores = np.abs((data - data.mean()) / data.std())
    return z_scores > threshold

# identify outliers for each column
outliers = data.apply(detect_outliers_zscore)

# count the number of outliers in each column
outlier_counts = outliers.sum()

# print the number of outliers for each column
print(outlier_counts)

* There are missing values for some crops: Winter Wheat has only 3037 non-null values out of 25017, Spring Rye has only 805 non-null values out of 25017, and Tame Hay has only 4205 non-null values out of 25017.
* The mean yield for the different crops ranges from 1.18 (Tame Hay) to 1408.06 (Chickpeas), the standard deviation of the yield for the different crops ranges from 0.65 (Tame Hay) to 579.64 (Chickpeas).
* For most crops, the median yield is close to the mean, indicating that the distribution is roughly symmetric.
* There are some crops with a large difference between the mean and median yield, indicating that the distribution may be skewed. For example, Mustard has a mean yield of 844.19 and a median yield of 847, suggesting that the distribution may be slightly skewed to the left.
* For some crops, such as Canary Seed and Chickpeas, the maximum yield is much higher than the 75th percentile, indicating the presence of outliers in the data.
* Overall, the dataset appears to have a wide range of yield values, with some crops having high variability in yield and potential outliers that may need to be addressed during data cleaning and preprocessing.

In [ ]:
# calculate skewness and kurtosis for each column
for col in data.columns[2:]:
    skewness = skew(data[col])
    kurt = kurtosis(data[col])
    print(f"{col} - Skewness: {skewness:.2f} Kurtosis: {kurt:.2f}")

#### *Data Exploration*

##### *Univariate Analysis*

In [ ]:
# Select only the columns with crop data
crop_cols = data.columns[2:]

# Create subplots with two plots in each row
fig, axs = plt.subplots(len(crop_cols)//2 + 1, 2)

# Flatten the axes array
axs = axs.flatten()

# Loop through the crop columns and plot a histogram for each crop
for i, col in enumerate(crop_cols):
    ax = axs[i]
    sns.histplot(data=data[col], ax=ax, kde=True)
    ax.set_title(col)
    ax.set_xlabel('Yield')
    ax.set_ylabel('Frequency')

# Remove the unused subplots
for ax in axs[len(crop_cols):]:
    ax.remove()

# Adjust the layout
fig.tight_layout()

# Show the plot
plt.show()

##### *Draw scatter plots for the relationship between 'Year' and each crop.*

In [ ]:
# Create a 4x4 subplot grid for the plots
fig, axs = plt.subplots(nrows=8, ncols=2)

# Loop over the crops and create a scatterplot for each one
for i, crop in enumerate(crops):
    row = i // 2
    col = i % 2
    axs[row, col].scatter(data['Year'], data[crop])
    axs[row, col].set_title(crop)

# Adjust the spacing between the plots and display the figure
fig.tight_layout()
plt.show()

##### *Draw scatter plots for the relationship between 'RM' and each crop.*

In [ ]:
# Create a 4x4 subplot grid for the plots
fig, axs = plt.subplots(nrows=8, ncols=2)

# Loop over the crops and create a scatterplot for each one
for i, crop in enumerate(crops):
    row = i // 2
    col = i % 2
    axs[row, col].scatter(data['RM'], data[crop])
    axs[row, col].set_title(crop)

# Adjust the spacing between the plots and display the figure
fig.tight_layout()
plt.show()

##### *Bivariatie Analysis*

In [ ]:
# Select numeric columns
crops = data.columns[2:]

# Create a correlation matrix
corr_matrix = data[crops].corr()

# Visualize the correlation matrix as a heatmap
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True)

###### *TBD*
In the case of 'Oats' and 'Spring Wheat', a strong positive relationship suggests that when the yield of 'Oats' is high in a particular year, the yield of 'Spring Wheat' is also likely to be high in that same year.

There are a number of possible reasons for this positive relationship. For example, both crops might have similar environmental requirements, such as soil type, temperature, or precipitation, which could affect their yields in similar ways. Alternatively, farmers may have similar planting, fertilization, or harvesting practices for both crops, which could also contribute to the observed relationship. It's also possible that there are other unmeasured factors that affect both crops, such as pests or diseases, that are driving the relationship.

#### *Data Transformation* 
Before moving to the next stage, we standardized the data by scaling the values of the crop yield data between 0 and 1. This was done to ensure that all features have equal weightage in our machine learning model.

##### *Standardization*

In [ ]:
# create a StandardScaler object
standard_scaler = StandardScaler()

# fit and transform the data using the scaler
data_standardized = standard_scaler.fit_transform(data)

data_standardized

In [ ]:
np.info(data_standardized)

##### *Normalization*

In [ ]:
# create a MinMaxScaler object
normalize_scaler = MinMaxScaler()

# fit and transform the standardized dataset using the scaler object
data_normalized = pd.DataFrame(normalize_scaler.fit_transform(data_standardized), columns=data.columns)

# display the normalized dataset
data_normalized

## III. Exploratory Data Analysis (EDA)

#### *Data Visualization*

##### *Summarize the total yield of crops in each Rural Municipal*

In [ ]:
data['TotalYield'] = data.iloc[:,2:].sum(axis=1)
data

In [ ]:
dtByYear = data.groupby('Year').sum()
dtByYear

In [ ]:
dfByRM = data.groupby('RM').sum()
dfByRM

In [ ]:
dfByRM_merged = dfByRM.merge(gdata, how='inner', on='RM')
dfByRM_merged

In [ ]:
dfByRM_merged_gdata = gpd.GeoDataFrame(dfByRM_merged)
# Plot choropleth map
fig, ax = plt.subplots()
dfByRM_merged_gdata.plot(column='TotalYield', cmap='YlGnBu', legend=True, ax=ax)
ax.set_title('Crop Yield by RM')
plt.show()

In [ ]:
dfByRM_merged_gdata.explore(
    column='TotalYield',
    cmap='inferno'
)

In [ ]:
# create a list of crop names
crop_names = list(data_normalized.columns)[2:]

# set up the plot
fig, axs = plt.subplots(8, 2)

# Loop through the crops and create a line plot for each one
for i, crop in enumerate(crop_names):
    # Create a subplot for each crop, with two crops per row
    plt.subplot(8, 2, i+1)
    
    # Set the title and axis labels
    plt.title(crop)
    plt.xlabel('Year')
    plt.ylabel('Yield')
    
    # Plot the line plot for the crop
    sns.lineplot(x='Year', y=crop, data=data_normalized)

# Adjust the spacing between subplots
plt.tight_layout()

# Show the plots
plt.show()



In [ ]:
# Select only the columns with crop data
crop_cols = data_normalized.columns[2:]

# Create subplots with two plots in each row
fig, axs = plt.subplots(len(crop_cols)//2 + 1, 2)

# Flatten the axes array
axs = axs.flatten()

# Loop through the crop columns and plot a histogram for each crop
for i, col in enumerate(crop_cols):
    ax = axs[i]
    sns.histplot(data=data_normalized[col], ax=ax, kde=True)
    ax.set_title(col)
    ax.set_xlabel('Yield')
    ax.set_ylabel('Frequency')

# Remove the unused subplots
for ax in axs[len(crop_cols):]:
    ax.remove()

# Adjust the layout
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Select numeric columns
numeric_cols = data_normalized.columns[2:]

# Create a correlation matrix
corr_matrix = data_normalized[numeric_cols].corr()

# Visualize the correlation matrix as a heatmap
sns.heatmap(corr_matrix, cmap='YlOrRd', annot=True)

#### *Feature Engineering*

In [ ]:
scaled_data_by_year = data_normalized.groupby('Year', as_index=False).mean()
scaled_data_by_year

In [ ]:
sns.lineplot(data=scaled_data_by_year[scaled_data_by_year.columns[2:]])

In [ ]:
sns.lineplot(data=scaled_data_by_year['Barley'])

In [ ]:
sns.lineplot(data=scaled_data_by_year['Canola'])

## IV. Machine Learning Model Development

#### *Model Selection*

Time Series Analysis: Since you only have crop yield data and no other relevant features, time series analysis would be a suitable approach to predict crop yields over time. You can use techniques like ARIMA, SARIMA, or LSTM to predict the yield of a specific crop over a period of time.

Clustering: Since you have yield data for 16 crops, you can use clustering techniques to group the crops that have similar yield patterns. This can help in identifying the factors that are contributing to the yield of each group of crops and thus, optimize the crop yield in the future.

#### *Model Training and Testing - Time Series Analysis*

In [ ]:
crop_name = 'Canola'
# Define the order of the ARIMA model
p = 1  # order of autoregressive term
d = 0  # degree of differencing
q = 1  # order of moving average term

# Fit the ARIMA model to the data
arima_model = ARIMA(data[crop_name], order=(p, d, q)).fit()

# Print a summary of the model
print(arima_model.summary())

In [ ]:
# assuming you have already trained your ARIMA model and have it stored as `model`

# make predictions for the next 5 time steps
predictions = arima_model.forecast(steps=5)

# print the predicted values
print(predictions)

In [ ]:

# Define the order of the ARIMA model
p = 1  # order of autoregressive term
d = 0  # degree of differencing
q = 1  # order of moving average term

# Fit the ARIMA model to the data
arima_model_scaled = ARIMA(scaled_data_by_year[crop_name], order=(p, d, q)).fit()

# Print a summary of the model
print(arima_model_scaled.summary())


In [ ]:
# assuming you have already trained your ARIMA model and have it stored as `model`

# make predictions for the next 5 time steps
predictions_scaled = arima_model_scaled.forecast(steps=5)

# print the predicted values
print(predictions_scaled)

In [ ]:
scaled_data_by_year[crop_name].reset_index()

In [ ]:
# plot the original data and the predicted values
sns.lineplot(data = scaled_data_by_year[crop_name])
sns.lineplot(data = predictions_scaled)
plt.show()

#### *Crop Yield clustering using K-Means clustering*

In [ ]:
# Create an empty list to store the WCSS values
wcss = []

# Loop over different values of k and perform k-means clustering
for i in range(1, 16):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(data_normalized)
    wcss.append(kmeans.inertia_)

# Plot the WCSS values against the values of k
plt.plot(range(1, 16), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# select the features to use in clustering
X = data_normalized.iloc[:, 2:].values

# set the number of clusters
k = 4

# fit the KMeans model
kmeans = KMeans(n_clusters=k, init='k-means++', max_iter=300, n_init=10, random_state=0)
y_kmeans = kmeans.fit_predict(X)

# plot the clusters
sns.set_style('whitegrid')
sns.scatterplot(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], color='blue', label='Cluster 1')
sns.scatterplot(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], color='green', label='Cluster 2')
sns.scatterplot(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], color='red', label='Cluster 3')
sns.scatterplot(X[y_kmeans == 3, 0], X[y_kmeans == 3, 1], color='orange', label='Cluster 4')
sns.scatterplot(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], color='black', label='Centroids')
plt.title('Crop Yield Clusters')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

In [ ]:
# Define the number of clusters (groups) you want to create
n_clusters = 4

# Create an instance of KMeans with the desired number of clusters
kmeans1 = KMeans(n_clusters=n_clusters, random_state=42)

# Fit the KMeans model to your scaled crop yield data
y1_kmeans = kmeans1.fit(data_normalized.iloc[:, 2:])

# Get the labels assigned to each data point (crop) by KMeans
labels1 = kmeans1.labels_

# Print out the value of all groups
for i in range(n_clusters):
    group_crops = data_normalized.iloc[:, 1][labels1 == i].tolist()
    #print(f"Group {i}: {group_crops}")
    print(f"Group {i}: {np.size(group_crops)}")


In [ ]:
# Create an empty list to store the WCSS values
wcss_rm = []

# Loop over different values of k and perform k-means clustering
for i in range(1, 16):
    kmeans_rm = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans_rm.fit(dfByRM)
    wcss_rm.append(kmeans_rm.inertia_)

# Plot the WCSS values against the values of k
plt.plot(range(1, 16), wcss_rm)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Define the number of clusters (groups) you want to create
n_clusters_rm = 4

# Create an instance of KMeans with the desired number of clusters
kmeans_rm = KMeans(n_clusters=n_clusters_rm, random_state=42)

# Fit the KMeans model to your scaled crop yield data
y_kmeans_rm = kmeans_rm.fit(dfByRM.iloc[:, 2:])

# Get the labels assigned to each data point (crop) by KMeans
labels_rm = kmeans_rm.labels_

# Print out the value of all groups
for i in range(n_clusters_rm):
    group_crops = dfByRM.iloc[:, 1][labels_rm == i].tolist()
    #print(f"Group {i}: {group_crops}")
    print(f"Group {i}: {np.size(group_crops)}")